# 作業工程計画

使用データ：FMT修正版_CRLEAデータ①入力様式_秋後半_v2_pretreated_v1.xlsx<br>
目的：予測値の誤差を集計する<br>
学習：検証以外の採点回<br>
検証：ある年度のある採点回<br>

0. ライブラリを読み込む
1. 整形データを読み込む
2. 整形データに学習・検証を分ける変数を追加する
3. 各組み合わせで予測値と、集計値を出す
4. 結果を結合して出力する

## ライブラリ

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import re 

"""
機械学習ライブラリの準備
"""

from sklearn.model_selection import cross_val_score
# from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeRegressor #決定木
import lightgbm as lgb #lightGBM

from sklearn import tree
import graphviz

### 村上さん pandas_tool

In [4]:
"""
村上さんtoolbox
"""
# pandas 基礎集計クラス
import numpy as np
import pandas as pd
import copy
import seaborn as sns
from itertools import combinations

#http://qiita.com/tanemaki/items/2ed05e258ef4c9e6caac

# Jupyterで表示するためには、最初に以下を実行すること
%matplotlib inline

# Static Classで設計する
class pandas_tool:
    
    # All in one チェック（Jupyterのみ）
    def all_basic_summary_jupyter(df):
        print("■ 型の確認")
        display(pandas_tool.type(df))
        print("■ 数値型の確認")
        display(pandas_tool.summary(df)[0])
        print("■ カテゴリ型の確認")
        cate_var_data = list(df.select_dtypes(include=['object']).columns)
        ret = pandas_tool.freq(df,cate_var_data)
        for d in ret:
            display(pd.DataFrame(d))
            print("---------------")
        print("■ 欠損の確認")
        display(pandas_tool.check_missing(df))
    
    # 相関関係可視化（Jupyterのみ）
    def all_value_relation_visualize(df):
        #sns.set_context("poster", 1.2, {"lines.linewidth": 3})
        sns.pairplot(df,size=5)
    
    # カテゴリ変数でのヒートマップ（Jupyterのみ）
    def make_heatmap(df,x,y,value):
        target_df = df.pivot_table(index=x,values=value,columns=y)
        sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues')
    
    # 散布図（Jupyterのみ）
    def make_scatter_chart(df,x,y):
        #sns.jointplot(x=x, y=y, data=df, kind="hex")
        sns.jointplot(x=x, y=y, data=df)
    
    # 組み合わせでヒートマップを作成（Jupyterのみ）
    def all_make_heatmap(df,var_list,value):
        col_num = 2
        var_list_set = list(combinations(var_list,2))
        
        fig, axes = plt.subplots(int(len(var_list_set)/col_num)+1, col_num, figsize=(18,3+6.5*int(len(var_list_set)/col_num)))
        
        for i,target in enumerate(var_list_set):
            target_df = df.pivot_table(index=target[0],values=value,columns=target[1])
            sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues', ax=axes[int(i/col_num), i%col_num])
            
        plt.tight_layout()
    
    # 数値集計
    def summary(df,view=False):
        ret=df.describe()
        mis_ret=df.isnull().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print("・統計量")
            print(ret)
            print("・欠損値")
            print(mis_ret)
            pd.set_option("display.max_columns",param)
        return ret,mis_ret
    
    # 型チェック
    def type(df,view=False):
        ret = df.dtypes
        if view:
            param=pd.get_option("display.max_rows")
            pd.set_option("display.max_rows",1000)
            print(ret)
            pd.set_option("display.max_rows",param)
        return ret
    
    # 欠損チェック
    def check_missing(df,view=False):
        not_null_df=df.notnull()
        ret=pd.DataFrame()
        for name in not_null_df.columns:
            tmp_df=not_null_df[name].value_counts()
            tmp_df.name=name
            ret = pd.concat([ret,tmp_df],axis=1)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 欠損値のオブザベーションを抽出
    def get_miss_data(df,column,view=False):
        ret=df[df[column].isnull()]
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        return ret
    
    # 欠損値を中央値で補完
    def fill_miss_med(df,var_name):
        var=df[var_name].median()
        df[var_name].fillna(var,inplace=True)
        return df
    
    # 欠損値を0で補完
    def fill_miss_zero(df,var_name):
        df[var_name].fillna(0,inplace=True)
        return df
    
    # 特定の値を欠損とみなす
    def apply_miss_value(df,var_name,value):
        df[var_name]=df[var_name].replace(value,np.nan)
        return df
    
    # 重複チェック
    def check_dup(df,columns,view=False):
        ret=pd.DataFrame()
        for name in columns:
            dup_cnt=df[name].duplicated().sum()
            tmp_df = pd.DataFrame({'var_name':[name],'dup_cnt':[dup_cnt]})
            ret = pd.concat([ret,tmp_df],axis=0,ignore_index= True)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 組み合わせ重複チェック
    def check_dup_comb(df,columns,view=False):
        ret = df[columns].duplicated().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # ユニークデータ取得
    def get_uniq_data(df,uniq_key,sort_key,keep='first'):
        ret = df.sort_values(by=sort_key)
        ret.drop_duplicates(subset=uniq_key, keep=keep, inplace=True)
        return ret
    
    # カテゴリ集計
    def freq(df,columns,view=False):
        ret=list()
        for name in columns:
            tmp_df=df[name].value_counts()
            tmp_df.name=name
            #ret = pd.concat([ret,tmp_df],axis=1)
            ret.append(tmp_df)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            for r in ret:
                print(r)
                #display(r)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 複雑な集計
    def tabulate(df,row,col=None,var='',func=np.sum,view=False):
        if var == '':
            tmp_df=df.reset_index(drop=False,inplace=False)
            ret=pd.pivot_table(data=tmp_df, values='index', index=row, columns=col, aggfunc='count', dropna=False, fill_value=0 ,margins = False)
            tmp_df=None
        else:
            ret=pd.pivot_table(data=df, values=var, index=row, columns=col, aggfunc=func, dropna=False, fill_value=0 ,margins = False)
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # マージ
    def merge(df1,df2,key,how,view=True):
        if view:
            print("df1のキー重複")
            pandas_tool.check_dup_comb(df1,key,True)
            print("df2のキー重複")
            pandas_tool.check_dup_comb(df2,key,True)
            
            print("df1のオブザベーション:{0}".format(len(df1)))
            print("df2のオブザベーション:{0}".format(len(df2)))
        
        ret=pd.merge(df1,df2,how=how,on=key)
        
        if view:
            print("mergeのオブザベーション:{0}".format(len(ret)))
        
        return ret
    
    # Rank
    def rank(df,var,num,suffix='_rank',check=False):
        labels=[i for i in range(0,num)]
        df[var+suffix]=pd.qcut(df[var], num, labels=labels)
        
        # check data
        if check:
            ret=pd.DataFrame()
            max_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.max,view=False)
            max_df.name='max'
            min_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.min,view=False)
            min_df.name='min'
            cnt_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func='count',view=False)
            cnt_df.name='count'
            ret=pd.concat([ret,min_df,max_df,cnt_df],axis=1)
            return df,ret
            
        return df
    
    # Rank適用(min基準)
    def apply_rank(df,rank_df):
        tmp_df=copy.deepcopy(rank_df)
        tmp_df.reset_index(drop=False,inplace=True)
        target_name=tmp_df.columns[3]
        tmp_df.columns=["rank","min","max","cnt"]
        
        def judge_thld(row):
            ret_var = -1
            cond_list = ["if 0 : ret_var = 0"]
            
            for i in range(1,len(tmp_df)):
                cond_list.append("elif row < " +str(tmp_df.ix[i,'min'])+ " : ret_var = " + str(tmp_df.ix[i-1,'rank']))
            
            cond_list.append("else: ret_var = " + str(tmp_df.ix[len(tmp_df)-1,'rank']))
            cond_str="\r\n".join(cond_list)
            # ローカル辞書をexecと共有する
            local_dict=locals()
            exec(cond_str,local_dict)
            return local_dict["ret_var"]
        
        df[target_name+"_rank"]=df[target_name].apply(judge_thld)
        return df
    
    # Min%以下はMin%点に、Max%以上はMax%点にクリップする
    def clip_min_max(df,col_list,apply_df=None,max_pct=0.99,min_pct=0.01):
        p_min = df[col_list].quantile(min_pct)
        p_max = df[col_list].quantile(max_pct)
        
        df[col] = df[col_list].clip(p_min,p_max,axis=1)
        
        # もしも適用先のデータがあるならば（例えば検証データ）対応
        if apply_df is not None:
            apply_df[col] = apply_df[col_list].clip(p_min,p_max,axis=1)
            return df,apply_df
        else:
            return df
    
    
    # 文字列→数値変換
    def conv_float(df,column,percent_flg=False):
        
        def conv_f(row):
            if row[column] == "" or row[column] is np.nan:
                return np.nan
            else:
                return float(row[column])
        
        df[column]=df[column].str.replace("\\","").str.replace(",","").str.replace("%","").str.strip()
        df[column]=df.apply(conv_f,axis=1)
        
        if percent_flg:
            df[column]=df[column]/100
        
        return df

## データの読み込み

In [5]:
#エクセルを読み込み、シートごとにデータフレームに入れる
"""
path_name = ファイルパス
sn_list = エクセルシート名のリスト
"""
path_name = "/Users/s.ogura/Documents/CRLEA/data/intermediatedata/FMT修正版_CRLEAデータ①入力様式_秋後半_v2_pretreated_v1.xlsx"
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names
#sn_list ['国語', '数学','英語']

# Excelファイルの読み込み
df_jpn = pd.read_excel(path_file, sn_list[0], header=0, index_col=None,skiprows=None,dtype = {'ロット番号': object})#headerを修正
df_math = pd.read_excel(path_file, sn_list[1], header=0, index_col=None,skiprows=None,dtype = {'ロット番号': object})#headerを修正
df_eng = pd.read_excel(path_file, sn_list[2], header=0, index_col=None,skiprows=None,dtype = {'ロット番号': object})#headerを修正

In [18]:
#train,test,predのデータ分類ラベル あとで試す。
def make_datalabel(df_input):
    df = df_input.copy()
    conditions = [
        (df['年度'] == 2018)&(df['採点回'] == '春'),#0
        (df['年度'] == 2018)&(df['採点回'] == '夏'),#1
        (df['年度'] == 2018)&(df['採点回'] == '秋'),#2
        (df['年度'] == 2018)&(df['採点回'] == '冬'),#3
        (df['年度'] == 2019)&(df['採点回'] == '春'),#4
        (df['年度'] == 2019)&(df['採点回'] == '夏'),#5
        (df['年度'] == 2019)&(df['採点回'] == '秋'),#6
        (df['年度'] == 2019)&(df['採点回'] == '冬'),#7
        (df['年度'] == 2020)&(df['採点回'] == '春'),#8
        (df['年度'] == 2020)&(df['採点回'] == '夏'),#9
        (df['年度'] == 2020)&(df['採点回'] == '秋'),#10
        (df['年度'] == 2020)&(df['採点回'] == '冬'),#11
        (df['年度'] == 2021)&(df['採点回'] == '春'),#12
        (df['年度'] == 2021)&(df['採点回'] == '夏'),#13
        (df['年度'] == 2021)&(df['採点回'] == '秋'),#14
        (df['年度'] == 2021)&(df['採点回'] == '冬'),#15
        (df['年度'] == 2022)&(df['採点回'] == '春'),#16
        (df['年度'] == 2022)&(df['採点回'] == '夏'),#17
        (df['年度'] == 2022)&(df['採点回'] == '秋')]#18
    choices = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
    df.loc[:,'data'] = np.select(conditions, choices, default = np.nan)
    
    conditions = [
        (df['年度'] == 2018)&(df['採点回'] == '春'),#0
        (df['年度'] == 2018)&(df['採点回'] == '夏'),#1
        (df['年度'] == 2018)&(df['採点回'] == '秋'),#2
        (df['年度'] == 2018)&(df['採点回'] == '冬'),#3
        (df['年度'] == 2019)&(df['採点回'] == '春'),#4
        (df['年度'] == 2019)&(df['採点回'] == '夏'),#5
        (df['年度'] == 2019)&(df['採点回'] == '秋'),#6
        (df['年度'] == 2019)&(df['採点回'] == '冬'),#7
        (df['年度'] == 2020)&(df['採点回'] == '春'),#8
        (df['年度'] == 2020)&(df['採点回'] == '夏'),#9
        (df['年度'] == 2020)&(df['採点回'] == '秋'),#10
        (df['年度'] == 2020)&(df['採点回'] == '冬'),#11
        (df['年度'] == 2021)&(df['採点回'] == '春'),#12
        (df['年度'] == 2021)&(df['採点回'] == '夏'),#13
        (df['年度'] == 2021)&(df['採点回'] == '秋'),#14
        (df['年度'] == 2021)&(df['採点回'] == '冬'),#15
        (df['年度'] == 2022)&(df['採点回'] == '春'),#16
        (df['年度'] == 2022)&(df['採点回'] == '夏'),#17
        (df['年度'] == 2022)&(df['採点回'] == '秋'),#16
        (df['年度'] == 2022)&(df['採点回'] == '冬')]#18
    choices = ['18春','18夏','18秋','18冬','19春','19夏','19秋','19冬',
               '20春','20夏','20秋','20冬','21春','21夏','21秋','21冬',
               '22春','22夏','22秋','22冬']
    df.loc[:,'ラベル'] = np.select(conditions, choices, default = np.nan)
    return df

In [19]:
df_jpn = make_datalabel(df_jpn)
df_math = make_datalabel(df_math)
df_eng = make_datalabel(df_eng)

In [20]:
df_jpn = df_jpn[~(df_jpn['data']==18)]
df_math = df_math[~(df_math['data']==18)]
df_eng = df_eng[~(df_eng['data']==18)]

In [21]:
df_jpn['ラベル'].unique()

array(['22夏', '21冬', '21秋', '21夏', '20冬', '20秋', '20夏', '19冬', '19秋',
       '19夏', '18冬', '18秋', '18夏'], dtype=object)

In [22]:
df_math['ラベル'].unique()

array(['22夏', '21夏', '21秋', '21冬', '21春', '20夏', '20秋', '20冬', '19夏',
       '19秋', '19冬', '18夏', '18秋', '18冬'], dtype=object)

In [23]:
df_eng['ラベル'].unique()

array(['22夏', '21冬', '21秋', '21夏', '20冬', '20秋', '20夏', '19冬', '19秋',
       '19夏', '18冬', '18秋', '18夏'], dtype=object)

## モデル作成

### DecisionTreeRegressorメソッドの概要

|引数名|概要|デフォルト|
| :---- | :---- | :---- |
|criterion|不純度を測定する基準（平均二乗誤差、平均絶対誤差など）|‘mse’|
|splitter|条件探索アルゴリズムを選択するオプション（’best’と’rondom’が指定可能）|‘best’|
|max_depth|決定木のノード深さの制限値。ツリーが深くなりすぎて過学習の状態に陥った際は、このパラメータが正則化の役割を果たす。|None|
|min_samples_split|ノードを分割するために必要なサンプルの最小値|2|
|min_samples_leaf|1ノードの深さを作成するために必要となるデータ数の最小値。指定した値以上のデータ数を持たないノードは作られない。|1|
|min_weight_fraction_leaf|サンプルの重みを考慮した上でのmin_samples_leafに該当|0.0|
|max_features|ランダムに指定する説明変数の数(全ての説明変数がモデル学習に活用されるわけではなく、ランダムに割り振られる）|None|
|random_state|乱数シード|None|
|max_leaf_nodes|作成される決定木の葉の数を、指定した値以下に制御する|None|
|min_impurity_decrease|決定木の成長の早期停止するための閾値。不純度が指定の値より減少した場合、ノードを分岐し、不純度が指定の値より減少しなければ分岐を抑制。|0.0|
|ccp_alpha|ccp_alphaが大きいほどプルーニングされるノードの数が増加。プルーニングとは、精度低下をできるだけ抑えながら過剰な重みを排除するプロセスを指す。|0.0|


## 関数定義

In [24]:
"""
https://stackoverflow.com/questions/50607740/reverse-a-get-dummies-encoding-in-pandas
ダミー変数を元に戻す。エクセル出力用
"""

def undummify(df, prefix_sep="_"):
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)
    return undummified_df

In [25]:
df_jpn.columns.values

array(['年度', '採点回', '学年', '科目', '分野', 'ロット番号', 'ポイント採点', '文字数', 'ポイント数',
       '配点要素の数', '配点', '必須条件の有無', '在宅勤務', '企画ペース', '最終ペース', 'data', 'ラベル'],
      dtype=object)

In [26]:
df_math.columns.values

array(['年度', '採点回', '学年', '科目', 'ロット番号', '（1）実点・立式点', '（1）カッコ点・立式点',
       '（1）実点・答え点', '（1）カッコ点・答え点', '（1）実点・その他', '（1）カッコ点・その他',
       '（2）実点・立式点', '（2）カッコ点・立式点', '（2）実点・答え点', '（2）カッコ点・答え点',
       '（2）実点・その他', '（2）カッコ点・その他', '（3）実点・立式点', '（3）カッコ点・立式点',
       '（3）実点・答え点', '（3）カッコ点・答え点', '（3）実点・その他', '（3）カッコ点・その他',
       '（1）ポイント数合計', '（2）ポイント数合計', '（3）ポイント数合計', 'ポイント採点', '（1）配点',
       '（2）配点', '（3）配点', '企画ペース', '最終ペース', '模範解答の記述行数（１）', '模範解答の記述行数（２）',
       '模範解答の記述行数（３）', '補足資料の枚数', '在宅勤務', '分野', 'data', 'ラベル'],
      dtype=object)

In [27]:
df_eng.columns.values

array(['年度', '採点回', '学年', '科目', 'ロット番号', '大問', '分野', '解答言語', 'ポイント採点',
       '企画ペース', '最終ペース', 'パターン数', '配点合計', '文字数合計', 'ポイント数合計', '減点要素の数合計',
       '在宅勤務', 'data', 'ラベル'], dtype=object)

In [29]:
def make_data(df_input, data = int, kamoku=str):#秋採点予測用前処理
    
    """
    国語モデル用の学習と検証データを作成
    1.データから不要な列を削除
    2.ダミー化
    3.説明変数と目的変数に分ける
    4.学習と検証に分ける
    5.選択用変数を除外し、各データフレームを作成
    6.[X_train,y_train,X_test,y_test,X_test_info]をデータフレームとしてreturn
    """
    df = df_input.copy()
    x = data
    if kamoku == 'jpn':
        #jpn
        #1.不要な列削除
        df = df.drop(columns = ["科目"])

        #2.名義変数のエンコーディング pandas get_dummies関数でone hot encording
        df = pd.get_dummies(df, columns=['分野'])
    if kamoku == 'math':
        #math
        #1.不要な列削除
        df = df.drop(columns = ['補足資料の枚数'])

        #2.名義変数のエンコーディング pandas get_dummies関数でone hot encording
        df = pd.get_dummies(df, columns=['科目','分野'])
    if kamoku == 'eng':
        #1.不要な列削除
        df = df.drop(columns = ['大問','科目'])

        #2.名義変数のエンコーディング pandas get_dummies関数でone hot encording
        df = pd.get_dummies(df, drop_first=True, columns=['解答言語'])
        df = pd.get_dummies(df, columns=['分野'])
    
    """
    #2.1 予測DFと学習検証DFの作成
    df_pred = df.copy()  
    df_pred= df_pred[df_pred['data'] == x]#納品する用の予測値算出に使うデータ。納品データの型にも使う。
    df = df[~(df['data'] == x)]#モデル作成と検証に使うデータ。22秋を除いて作成。
    """
    #3.上記のデータセットを説明変数と目的変数で分ける
    df_X = df.drop(columns = {"最終ペース"})#目的変数を除外した（説明変数と選択用変数だけ含む）データフレーム
    df_y = df.loc[:,['最終ペース','data']]#目的変数と選択用変数だけ含むデータフレーム
    
    #4.データフレームの分離
    """訓練用"""
    #訓練用説明変数
    X_train = df_X[~(df_X['data'] == x)]
    #訓練用目的変数
    y_train = df_y[~(df_y['data'] == x)]

    """検証用"""
    #訓練用説明変数
    X_test = df_X[df_X['data'] == x]
    #訓練用目的変数
    y_test = df_y[df_y['data'] == x]
    
    """納品用"""
    """
    #訓練用説明変数
    X_pred = df_pred.drop(columns = {"最終ペース","企画ペース",'採点回',"年度",'ロット番号','data'})#目的変数を除外した（説明変数と選択用変数だけ含む）データフレーム
    #訓練用目的変数
    #y_pred = df_pred.loc[:,['年度','採点回_夏','最終ペース']]#目的変数と選択用変数だけ含むデータフレーム
    """
    
    #5.不要な変数を削除
    rem_cols_x = ["企画ペース","採点回","年度","ロット番号",'ラベル','data']#結果出力で使うけど学習には使わない
    rem_cols_y = ['data']

    #説明変数
    X_train = X_train.drop(columns = rem_cols_x)
    X_test_kikaku =X_test.loc[:,['企画ペース']]
    X_test = X_test.drop(columns = rem_cols_x)
         
    #目的変数
    y_train = y_train.drop(columns = rem_cols_y)
    y_test = y_test.drop(columns = rem_cols_y)
    
    #各モデル用データを各データフレームにまとめ、それらをさらにデータフレームにまとめて返す。
    # df_res=[X_train,y_train,X_test,y_test,X_test_kikaku,X_pred,df_pred]
    df_res=[X_train,y_train,X_test,y_test,X_test_kikaku]

    return df_res

In [30]:
"""
モデル作成
"""
def train_model(df,a=2,b=1,c=None):
    X_train= df[0]
    y_train= df[1]
    model = DecisionTreeRegressor(criterion='mse', 
                                   splitter='best', 
                                   max_depth=c, 
                                   min_samples_split=a, #3,4,5とか？
                                   min_samples_leaf=b,#2とか 
                                   min_weight_fraction_leaf=0.0,
                                   max_features=None, 
                                   random_state=0,#出力結果の固定のため 
                                   max_leaf_nodes=None, 
                                   min_impurity_decrease=0.0, 
                                   ccp_alpha=0.0
                                  )

    #上記のパラメータでモデルを学習する
    model.fit(X_train, y_train)
    return model

In [31]:
def make_past_summary(X_train,y_train): 
    """
    学習データを使って分野別の集計値を返す
    """
    #X_trainとy_trainをconcatしたdfを作る
    df = pd.concat([X_train, y_train], axis=1)
    #dfをundemmify
    df = undummify(df)

    group_df1 = df[['分野','学年','最終ペース']].groupby(['分野','学年']).mean().round(1).rename(columns={"最終ペース":"学習データ_平均値"})
    group_df1.reset_index(inplace=True)

    group_df2 = df[['分野','学年','最終ペース']].groupby(['分野','学年']).max().round(1).rename(columns={"最終ペース":"学習データ_最大値"})
    group_df2.reset_index(inplace=True)

    group_df3 = df[['分野','学年','最終ペース']].groupby(['分野','学年']).min().round(1).rename(columns={"最終ペース":"学習データ_最小値"})
    group_df3.reset_index(inplace=True)

    group_df4 = df[['分野','学年','最終ペース']].groupby(['分野','学年']).median().round(1).rename(columns={"最終ペース":"学習データ_中央値"})
    group_df4.reset_index(inplace=True)

    group_df5 = df[['分野','学年','最終ペース']].groupby(['分野','学年']).count().round(1).rename(columns={"最終ペース":"学習データ_N数"})
    group_df5.reset_index(inplace=True)


    # merge
    group_df = pd.merge(group_df1,group_df2, on = ['分野','学年'],how = 'left')
    group_df = pd.merge(group_df,group_df3, on = ['分野','学年'],how = 'left')
    group_df = pd.merge(group_df,group_df4, on = ['分野','学年'],how = 'left')
    group_df = pd.merge(group_df,group_df5, on = ['分野','学年'],how = 'left')
    #group_df = group_df.rename(columns={category:"分野"})
    return group_df

In [32]:
def get_train(df): 
    """
    学習データを結合したdfを返す
    """
    X_train= df[0]
    y_train= df[1]
    #X_trainとy_trainをconcatしたdfを作る
    df = pd.concat([X_train, y_train], axis=1)
    #dfをundemmify
    res = undummify(df)
    return res

In [87]:
def get_test(df,model):# 検証データの予測値を出し、使用した訓練データの過去集計値をくっつける.
    """
    モデルを使って、予測値を出し、予測値、最終ペース、乖離度（予測値/実測値）を列に追加したdfを返す。
    """
    X_train= df[0]
    y_train= df[1]
    X_test = df[2]
    y_test = df[3]
    X_test_kikaku = df[4]
    y_pred  = model.predict(X_test)
    
    #得た結果を学習データとマージしてデータフレームで返す
    df_res=[]
    df_res = undummify(X_test) 
    df_res.loc[:,'AI予測ペース']= y_pred #上のデータに予測値をマージ
    df_res.loc[:,'企画ペース']= X_test_kikaku
    df_res.loc[:,'最終ペース']= y_test
    df_res.loc[:,'AI予測/人予測']= df_res['AI予測ペース']/df_res['企画ペース']#AI予測値と人予測値の比率を列に追加 
    df_res.loc[:,'AI乖離度']= df_res['AI予測ペース']/df_res['最終ペース']#実測値と予測値の比率を列に追加 
    df_res.loc[:,'元の乖離度']= df_res['企画ペース']/df_res['最終ペース']#実測値と予測値の比率を列に追加       
    # 最終ペースをランク分け
    x = "最終ペース"
    conditions = [
        (df_res[x] >= 600),
        (df_res[x] >= 500),
        (df_res[x] >= 400),
        (df_res[x] >= 300),
        (df_res[x] >= 200),
        (df_res[x] >= 100),
        (df_res[x] >= 0)
         ]
    choices = ["600~", "500~600", "400~500", "300~400", "200~300", "100~200","0~100"]
    df_res.loc[:,'最終ランク'] = np.select(conditions, choices, default = 0)
    
    # 最終ペースをランク分け
    x = "最終ペース"
    conditions = [
        (df_res[x] >= 600),
        (df_res[x] >= 550),
        (df_res[x] >= 500),
        (df_res[x] >= 450),
        (df_res[x] >= 400),
        (df_res[x] >= 350),
        (df_res[x] >= 300),
        (df_res[x] >= 250),
        (df_res[x] >= 200),
        (df_res[x] >= 150),
        (df_res[x] >= 100),
        (df_res[x] >= 50),
        (df_res[x] >= 0)
         ]
    choices = ["600~", "550~600", "500~550", "450~500", "400~450", "350~400","300~350",
               "250~300", "200~250", "150~200", "100~150", "050~100","00〜50"]
    df_res.loc[:,'最終ランク2'] = np.select(conditions, choices, default = 0)
    
    #大外れの予測値に対してフラグ作成
    x = "AI予測/人予測"
    conditions = [
        (df_res[x] > 1.5),
        (df_res[x] < 0.5)
         ]
    choices = [1,1]
    df_res.loc[:,'flag_AI予測/人予測'] = np.select(conditions, choices, default = 0)

    x = "AI乖離度"
    conditions = [
        (df_res[x] > 1.5),
        (df_res[x] < 0.5)
         ]
    choices = [1,1]
    df_res.loc[:,'flag_AI予測/実測値'] = np.select(conditions, choices, default = 0)

    x = "元の乖離度"
    conditions = [
        (df_res[x] > 1.5),
        (df_res[x] < 0.5)
         ]
    choices = [1,1]
    df_res.loc[:,'flag_企画/実測値'] = np.select(conditions, choices, default = 0)
    
    df_ref = make_past_summary(X_train,y_train)
    df_res = pd.merge(df_res,df_ref, on = ['分野','学年'],how = 'left')
    return df_res

In [29]:
def get_pred(df,model):# 予測データの予測値を出し、使用した訓練データの過去集計値をくっつける.
    """
    モデルを使って、予測値を出し、予測値、最終ペース、差分、乖離度（予測値/最終ペース）を列に追加したdfを返す。
    """
    X_train= df[0]#訓練の説明変数
    y_train= df[1]#訓練の目的変数
    X_pred = df[5]#予測の説明変数
    df_pred = df[6]#予測の元データ。説明変数以外の情報を含む。
    y_pred  = model.predict(X_pred)
    
    #得た結果を学習データとマージしてデータフレームで返す
    df_res=[]
    df_res = undummify(df_pred) 
    df_res.loc[:,'AI想定ペース']= y_pred #上のデータに予測値をマージ
    #訓練データの統計量を作成して、結合
    df_ref = make_past_summary(X_train,y_train)
    df_res = pd.merge(df_res,df_ref, on = ['分野','学年'],how = 'left')
    return df_res

In [13]:
def get_importance(df,model):
    """
    変数の重要度を返す
    """
    X_test = df[2]
    feature = model.feature_importances_
    label = X_test.columns
    indices = np.argsort(feature)

    # 特徴量の重要度の棒グラフ
    fig =plt.figure (figsize = (10,10))
    plt.ion()
    plt.barh(range(len(feature)), feature[indices])
    plt.yticks(range(len(feature)), label[indices], fontsize=14)
    plt.xticks(fontsize=14)
    plt.ylabel("Feature", fontsize=18)
    plt.xlabel("Feature Importance", fontsize=18)
    plt.tight_layout()
    plt.close(fig)
    return fig

In [14]:
#使用ライブラリ　樹形図作成
# from sklearn import tree
# import graphviz
def make_treegraph(df,model):
    """
    樹形図を作成し、返す
    """
    X_train= df[0]
    X_train_feature_names = X_train.columns.values.tolist()
    dot_data = tree.export_graphviz(model, out_file=None,
                                    feature_names= X_train_feature_names,
                                    # class_names=iris.target_names,
                                    filled=True, rounded=True, special_characters=True
                                    )
    treegraph = graphviz.Source(dot_data) 
    return treegraph

In [34]:
#機械学習の実行関数
def get_result(df_input,data,kamoku):
    df = df_input.copy()
    df_pret = make_data(df,data,kamoku)#データマートをML用に整形
    model = train_model(df_pret,a=2,b=1,c=None)#モデル訓練
    df_test = get_test(df_pret, model)#検証データ
    df_res = df_test
    return df_res

In [43]:
#繰り返しの実行

def get_output(df_input,kamoku):
    df = df_input.copy()
    data_list = sorted(df['data'].unique())#リストを作成し昇順に並び替え
    df_res=[]
    for i in data_list:
        ret = get_result(df,i,kamoku)
        ret['data']=i #予測結果にdataをデータグループとしてつける
        df_res.append(ret)
    #集計結果を一つのDFに結合
    dfs = df_res[0]
    for i in df_res[1:]:
        dfs = dfs.append(i)
    #ラベル作成
    df_label= df[['data','ラベル']]
    df_label = df_label.drop_duplicates()
    #join
    df1 = dfs
    df2 = df_label
    result = pandas_tool.merge(df1,df2, key = ["data"],how = 'left')
    #企画ペースの欠損削除
    colname='企画ペース'
    result2 = result.dropna(subset=[colname])
    results =[result, result2]
    return results

## 結果出力

In [88]:
#予測値のDF作成
df_res_jpn = get_output(df_jpn,'jpn')

df1のキー重複
319
df2のキー重複
0
df1のオブザベーション:332
df2のオブザベーション:13
mergeのオブザベーション:332


In [89]:
#結果出力
select_cols = ['最終ペース','企画ペース','AI予測ペース','AI予測/人予測','AI乖離度','元の乖離度']
df_dm = df_jpn #データマート
df_res = df_res_jpn


name_file = "分野別ペース予測誤差_国語_1215_v3" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name_file)) as writer:
    df_dm.to_excel(writer, sheet_name='データマート',encoding='utf-8-sig', index = False)
    df_res[0].to_excel(writer, sheet_name='検証値',encoding='utf-8-sig', index = False)
    df_res[1].to_excel(writer, sheet_name='検証値_企画ペース0除去',encoding='utf-8-sig', index = False)
    pandas_tool.summary(df_res[0].loc[:,select_cols])[0].to_excel(writer, sheet_name='検証値要約',encoding='utf-8-sig', index = True)

In [90]:
#予測値のDF作成
df_res_math = get_output(df_math,'math')

df1のキー重複
449
df2のキー重複
0
df1のオブザベーション:463
df2のオブザベーション:14
mergeのオブザベーション:463


In [91]:
#結果出力
select_cols = ['最終ペース','企画ペース','AI予測ペース','AI予測/人予測','AI乖離度','元の乖離度']
df_dm = df_math #データマート
df_res = df_res_math


name_file = "分野別ペース予測誤差_数学_1215_v3" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name_file)) as writer:
    df_dm.to_excel(writer, sheet_name='データマート',encoding='utf-8-sig', index = False)
    df_res[0].to_excel(writer, sheet_name='検証値',encoding='utf-8-sig', index = False)
    df_res[1].to_excel(writer, sheet_name='検証値_企画ペース0除去',encoding='utf-8-sig', index = False)
    pandas_tool.summary(df_res[0].loc[:,select_cols])[0].to_excel(writer, sheet_name='検証値要約',encoding='utf-8-sig', index = True)

In [92]:
#予測値のDF作成
df_res_eng = get_output(df_eng,'eng')

df1のキー重複
339
df2のキー重複
0
df1のオブザベーション:352
df2のオブザベーション:13
mergeのオブザベーション:352


In [93]:
#結果出力
select_cols = ['最終ペース','企画ペース','AI予測ペース','AI予測/人予測','AI乖離度','元の乖離度']
df_dm = df_eng #データマート
df_res = df_res_eng


name_file = "分野別ペース予測誤差_英語_1215_v3" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name_file)) as writer:
    df_dm.to_excel(writer, sheet_name='データマート',encoding='utf-8-sig', index = False)
    df_res[0].to_excel(writer, sheet_name='検証値',encoding='utf-8-sig', index = False)
    df_res[1].to_excel(writer, sheet_name='検証値_企画ペース0除去',encoding='utf-8-sig', index = False)
    pandas_tool.summary(df_res[0].loc[:,select_cols])[0].to_excel(writer, sheet_name='検証値要約',encoding='utf-8-sig', index = True)

In [95]:
"""
csvの書き出し
"""
#保存先
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス

df = df_res_jpn[0]
name = "採点ペース_学習データ分布_国語_v1"
df.to_csv('{}/{}.csv'.format(path_folder,"{}".format(name)),encoding='utf-8-sig',index=False)

df = df_res_math[0]
name = "採点ペース_学習データ分布_数学_v1"
df.to_csv('{}/{}.csv'.format(path_folder,"{}".format(name)),encoding='utf-8-sig',index=False)

df = df_res_eng[0]
name = "採点ペース_学習データ分布_英語_v1"
df.to_csv('{}/{}.csv'.format(path_folder,"{}".format(name)),encoding='utf-8-sig',index=False)

# 予測値の集計

In [80]:
df_input = df_jpn
df_input2 = df_res_jpn[0]
kamoku = '国語'
#学習データの科目分野別基礎統計量を作る
df1 = df_input.copy()
df1g = df1[['分野','最終ペース']].groupby(['分野']).describe().round(1)

#予測値の科目分野別基礎統計量を作る
df2 = df_input2.copy()
df2g = df2[['分野','AI乖離度','flag_AI予測/実測値']].groupby(['分野']).describe().round(2)

#分野名で結合する
df_ret = pd.merge(df1g,df2g, on = ['分野'],how = 'left')
df_ret['科目']= kamoku

In [86]:
def make_stat_table(df_input, df_input2):
    #学習データの科目分野別基礎統計量を作る
    df1 = df_input.copy()
    df1g = df1[['分野','最終ペース']].groupby(['分野']).describe().round(1)

    #予測値の科目分野別基礎統計量を作る
    df2 = df_input2.copy()
    df2g = df2[['分野','AI乖離度','flag_AI予測/実測値']].groupby(['分野']).describe().round(2)

    #分野名で結合する
    df_ret = pd.merge(df1g,df2g, on = ['分野'],how = 'left')
    return df_ret

In [84]:
df_stat_jpn = make_stat_table(df_jpn,df_res_jpn[0])
df_stat_math = make_stat_table(df_math,df_res_math[0])
df_stat_eng = make_stat_table(df_eng,df_res_eng[0])

In [85]:
#結果出力
df1 = df_stat_jpn
df2 = df_stat_math
df3 = df_stat_eng

name_file = "分野別ペース予測誤差_予測集計_v2" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name_file)) as writer:
    df1.to_excel(writer, sheet_name='国語',encoding='utf-8-sig', index = True)
    df2.to_excel(writer, sheet_name='数学',encoding='utf-8-sig', index = True)
    df3.to_excel(writer, sheet_name='英語',encoding='utf-8-sig', index = True)

# メモ

In [24]:
def make_data_jpn(df_input, data = int):#秋採点予測用前処理
    
    """
    国語モデル用の学習と検証データを作成
    1.データから不要な列を削除
    2.ダミー化
    3.説明変数と目的変数に分ける
    4.学習と検証に分ける
    5.選択用変数を除外し、各データフレームを作成
    6.[X_train,y_train,X_test,y_test,X_test_info]をデータフレームとしてreturn
    """
    df = df_input.copy()
    x = data

    #1.不要な列削除
    df = df.drop(columns = ["科目"])

    #2.名義変数のエンコーディング pandas get_dummies関数でone hot encording
    df = pd.get_dummies(df, columns=['分野'])
    """
    #2.1 予測DFと学習検証DFの作成
    df_pred = df.copy()  
    df_pred= df_pred[df_pred['data'] == x]#納品する用の予測値算出に使うデータ。納品データの型にも使う。
    df = df[~(df['data'] == x)]#モデル作成と検証に使うデータ。22秋を除いて作成。
    """
    #3.上記のデータセットを説明変数と目的変数で分ける
    df_X = df.drop(columns = {"最終ペース"})#目的変数を除外した（説明変数と選択用変数だけ含む）データフレーム
    df_y = df.loc[:,['最終ペース','data']]#目的変数と選択用変数だけ含むデータフレーム
    
    #4.データフレームの分離
    """訓練用"""
    #訓練用説明変数
    X_train = df_X[~(df_X['data'] == x)]
    #訓練用目的変数
    y_train = df_y[~(df_y['data'] == x)]

    """検証用"""
    #訓練用説明変数
    X_test = df_X[df_X['data'] == x]
    #訓練用目的変数
    y_test = df_y[df_y['data'] == x]
    
    """納品用"""
    """
    #訓練用説明変数
    X_pred = df_pred.drop(columns = {"最終ペース","企画ペース",'採点回',"年度",'ロット番号','data'})#目的変数を除外した（説明変数と選択用変数だけ含む）データフレーム
    #訓練用目的変数
    #y_pred = df_pred.loc[:,['年度','採点回_夏','最終ペース']]#目的変数と選択用変数だけ含むデータフレーム
    """
    
    #5.不要な変数を削除
    rem_cols_x = ["企画ペース","採点回","年度","ロット番号",'data']#結果出力で使うけど学習には使わない
    rem_cols_y = ['data']

    #説明変数
    X_train = X_train.drop(columns = rem_cols_x)
    X_test_kikaku =X_test.loc[:,['企画ペース']]
    X_test = X_test.drop(columns = rem_cols_x)
         
    #目的変数
    y_train = y_train.drop(columns = rem_cols_y)
    y_test = y_test.drop(columns = rem_cols_y)
    
    #各モデル用データを各データフレームにまとめ、それらをさらにデータフレームにまとめて返す。
    # df_res=[X_train,y_train,X_test,y_test,X_test_kikaku,X_pred,df_pred]
    df_res=[X_train,y_train,X_test,y_test,X_test_kikaku]

    return df_res

In [30]:
#機械学習の実行関数
def get_result_jpn(df_input,data):
    df = df_input.copy()
    df_pret = make_data_jpn(df,data)#データマートをML用に整形
    model = train_model(df_pret,a=2,b=1,c=None)#モデル訓練
    df_test = get_test(df_pret, model)#検証データ
    df_res = df_test
    return df_res

In [64]:
#繰り返しの実行

def get_output(df_input):
    df = df_input.copy()
    data_list = sorted(df['data'].unique())#リストを作成し昇順に並び替え
    df_res=[]
    for i in data_list:
        ret = get_result_jpn(df,i)
        ret['data_group']=i #予測結果にdataをデータグループとしてつける
        df_res.append(ret)
    #集計結果を一つのDFに結合
    dfs = df_res[0]
    for i in df_res[1:]:
        dfs = dfs.append(i)
    colname='企画ペース'
    dfs2 = dfs.dropna(subset=[colname])
    df_ret =[dfs,dfs2]
    return df_ret

In [71]:
#予測値のDF作成
df_test = get_output(df_jpn)

In [72]:
"""
Excelの書き出し 
"""

name = "誤差分析_v1.1"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス

#Excel
with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name)) as writer:
    df_test[0].to_excel(writer, sheet_name='data',encoding='utf-8-sig', index = False)
    df_test[1].to_excel(writer, sheet_name='企画ペース空白削除',encoding='utf-8-sig', index = False)

In [73]:
#結果出力
select_cols = ['最終ペース','企画ペース','AI予測ペース','AI予測/人予測','AI乖離度','元の乖離度']
df_dm = df_jpn #データマート
df_res = df_test


name_file = "分野別ペース予測誤差_国語_1215_v1" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name_file)) as writer:
    df_dm.to_excel(writer, sheet_name='データマート',encoding='utf-8-sig', index = False)
    df_res[0].to_excel(writer, sheet_name='検証値',encoding='utf-8-sig', index = False)
    df_res[1].to_excel(writer, sheet_name='検証値_企画ペース0除去',encoding='utf-8-sig', index = False)
    pandas_tool.summary(df_res[0].loc[:,select_cols])[0].to_excel(writer, sheet_name='検証値要約',encoding='utf-8-sig', index = True)

In [7]:
"""
def make_data_jpn(df_input):#秋採点予測 orijinal
    
    """
    国語モデル用の学習と検証データを作成
    1.データから不要な列を削除
    2.ダミー化
    3.説明変数と目的変数に分ける
    4.学習と検証に分ける
    5.選択用変数を除外し、各データフレームを作成
    6.[X_train,y_train,X_test,y_test,X_test_info]をデータフレームとしてreturn
    """
    df = df_input.copy()

    #1.不要な列削除
    df = df.drop(columns = ["科目"])

    #2.名義変数のエンコーディング pandas get_dummies関数でone hot encording
    df = pd.get_dummies(df, columns=['分野'])
    df_pred = df.copy()
    df_pred= df_pred[(df_pred['採点回'] == '秋')&(df_pred['年度']==2022)]#納品する用の予測値算出に使うデータ。納品データの型にも使う。
    df = df[~((df['採点回'] == '秋')&(df['年度']==2022))]#モデル作成と検証に使うデータ。22秋を除いて作成。
    #3.上記のデータセットを説明変数と目的変数で分ける
    df_X = df.drop(columns = {"最終ペース"})#目的変数を除外した（説明変数と選択用変数だけ含む）データフレーム
    df_y = df.loc[:,['年度','採点回','最終ペース']]#目的変数と選択用変数だけ含むデータフレーム
    
    #4.データフレームの分離
    """秋冬が訓練用"""
    #訓練用説明変数
    X_train = df_X[~((df_X['採点回'] == '夏')&(df_X['年度']==2022))]
    #訓練用目的変数
    y_train = df_y[~((df_y['採点回'] == '夏')&(df_y['年度']==2022))]

    """22夏が検証用"""
    #訓練用説明変数
    X_test = df_X[(df_X['採点回'] == '夏')&(df_X['年度']==2022)]
    #訓練用目的変数
    y_test = df_y[(df_y['採点回'] == '夏')&(df_y['年度']==2022)]
    """22秋が納品用"""
    #訓練用説明変数
    X_pred = df_pred.drop(columns = {"最終ペース","企画ペース",'採点回',"年度",'ロット番号'})#目的変数を除外した（説明変数と選択用変数だけ含む）データフレーム
    #訓練用目的変数
    #y_pred = df_pred.loc[:,['年度','採点回_夏','最終ペース']]#目的変数と選択用変数だけ含むデータフレーム
    
    #5.不要な変数を削除
    rem_cols_x = ["企画ペース","採点回","年度","ロット番号"]
    rem_cols_y = ["採点回","年度"]

    #説明変数
    X_train = X_train.drop(columns = rem_cols_x)
    X_test_kikaku =X_test.loc[:,['企画ペース']]
    X_test = X_test.drop(columns = rem_cols_x)
         
    #目的変数
    y_train = y_train.drop(columns = rem_cols_y)
    y_test = y_test.drop(columns = rem_cols_y)
    
    #各モデル用データを各データフレームにまとめ、それらをさらにデータフレームにまとめて返す。
    df_res=[X_train,y_train,X_test,y_test,X_test_kikaku,X_pred,df_pred]

    return df_res
"""

In [76]:
#機械学習の実行
"""
df_pret_jpn = make_data_jpn(df_jpn)#旧変数で2020-2021
model_jpn = train_model(df_pret_jpn,a=2,b=1,c=None)
df_train=get_train(df_pret_jpn)
df_test = get_test(df_pret_jpn, model_jpn)
df_pred = get_pred(df_pret_jpn, model_jpn)
df_res_jpn=[df_train, df_test, df_pred]
"""

'\ndf_pret_jpn = make_data_jpn(df_jpn)#旧変数で2020-2021\nmodel_jpn = train_model(df_pret_jpn,a=2,b=1,c=None)\ndf_train=get_train(df_pret_jpn)\ndf_test = get_test(df_pret_jpn, model_jpn)\ndf_pred = get_pred(df_pret_jpn, model_jpn)\ndf_res_jpn=[df_train, df_test, df_pred]\n'

In [15]:
"""
#機械学習の実行関数
def get_result_jpn(df_input):
    df = df_input.copy()
    df_pret = make_data_jpn(df)#データマートをML用に整形
    model = train_model(df_pret,a=2,b=1,c=None)#モデル訓練
    df_train= get_train(df_pret)#訓練データ
    df_test = get_test(df_pret, model)#検証データ
    df_pred = get_pred(df_pret, model)#予測データ
    df_res=[df_pret, model, df_train, df_test, df_pred]#予測用
    return df_res
"""

In [16]:
#機械学習の実行
df_res_jpn = get_result_jpn(df_jpn)

In [17]:
#学習値と予測値の説明変数の列の並びが等しいか
#df_res[0]=[X_train,y_train,X_test,y_test,X_test_kikaku,X_pred,df_pred]
df = df_res_jpn
df[0][0].columns.values == df[0][5].columns.values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [18]:
#説明変数の確認
df_res_jpn[0][5].columns.values

array(['学年', 'ポイント採点', '文字数', 'ポイント数', '配点要素の数', '配点', '必須条件の有無', '在宅勤務',
       '分野_古文（内説）', '分野_古文（現訳）', '分野_小説', '分野_漢文（内説）', '分野_漢文（書下し）',
       '分野_漢文（現訳）', '分野_評論', '分野_随筆'], dtype=object)

In [19]:
#データの数
len(df_jpn)

361

In [20]:
#学習するデータの数
len(df_res_jpn[0][0])

307

In [21]:
#検証するデータの数
len(df_res_jpn[0][2])

25

In [22]:
#予測するデータの数
len(df_res_jpn[0][5])

29

In [23]:
"""
#結果出力
select_cols = ['最終ペース','企画ペース','AI想定ペース','実企誤差','実予誤差','AI乖離度','元の乖離度']
df=df_res_jpn


name_file = "分野別ペース予測誤差_国語_1215_v1" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name_file)) as writer:
    df[2].to_excel(writer, sheet_name='訓練値_国語',encoding='utf-8-sig', index = False)
    df[3].to_excel(writer, sheet_name='検証値_国語',encoding='utf-8-sig', index = False)
    pandas_tool.summary(df[3].loc[:,select_cols])[0].to_excel(writer, sheet_name='検証値要約_国語',encoding='utf-8-sig', index = True)
    df[4].to_excel(writer, sheet_name='予測値_国語',encoding='utf-8-sig', index = False)
"""

## 数学でモデルを作成

In [25]:
def make_data_math(df_input):#秋採点予測　orijinal
    
    """
    国語モデル用の学習と検証データを作成
    1.データから不要な列を削除
    2.ダミー化
    3.説明変数と目的変数に分ける
    4.学習と検証に分ける
    5.選択用変数を除外し、各データフレームを作成
    6.[X_train,y_train,X_test,y_test,X_test_info]をデータフレームとしてreturn
    """
    df = df_input.copy()

    #1.不要な列削除
    df = df.drop(columns = ['補足資料の枚数'])

    #2.名義変数のエンコーディング pandas get_dummies関数でone hot encording
    df = pd.get_dummies(df, columns=['科目','分野'])
    df_pred = df.copy()
    df_pred= df_pred[(df_pred['採点回'] == '秋')&(df_pred['年度']==2022)]#納品する用の予測値算出に使うデータ。納品データの型にも使う。
    df = df[~((df['採点回'] == '秋')&(df['年度']==2022))]#モデル作成と検証に使うデータ。22秋を除いて作成。
    #3.上記のデータセットを説明変数と目的変数で分ける
    df_X = df.drop(columns = {"最終ペース"})#目的変数を除外した（説明変数と選択用変数だけ含む）データフレーム
    df_y = df.loc[:,['年度','採点回','最終ペース']]#目的変数と選択用変数だけ含むデータフレーム
    
    #4.データフレームの分離
    """秋冬が訓練用"""
    #訓練用説明変数
    X_train = df_X[~((df_X['採点回'] == '夏')&(df_X['年度']==2022))]
    #訓練用目的変数
    y_train = df_y[~((df_y['採点回'] == '夏')&(df_y['年度']==2022))]

    """22夏が検証用"""
    #訓練用説明変数
    X_test = df_X[(df_X['採点回'] == '夏')&(df_X['年度']==2022)]
    #訓練用目的変数
    y_test = df_y[(df_y['採点回'] == '夏')&(df_y['年度']==2022)]
    """22秋が納品用"""
    #訓練用説明変数
    X_pred = df_pred.drop(columns = {"最終ペース","企画ペース",'採点回',"年度",'ロット番号'})#目的変数を除外した（説明変数と選択用変数だけ含む）データフレーム
    #訓練用目的変数
    #y_pred = df_pred.loc[:,['年度','採点回_夏','最終ペース']]#目的変数と選択用変数だけ含むデータフレーム
    
    #5.不要な変数を削除
    rem_cols_x = ["企画ペース","採点回","年度","ロット番号"]
    rem_cols_y = ["採点回","年度"]

    #説明変数
    X_train = X_train.drop(columns = rem_cols_x)
    X_test_kikaku =X_test.loc[:,['企画ペース']]
    X_test = X_test.drop(columns = rem_cols_x)
         
    #目的変数
    y_train = y_train.drop(columns = rem_cols_y)
    y_test = y_test.drop(columns = rem_cols_y)
    
    #各モデル用データを各データフレームにまとめ、それらをさらにデータフレームにまとめて返す。
    df_res=[X_train,y_train,X_test,y_test,X_test_kikaku,X_pred,df_pred]

    return df_res

## 結果出力

In [26]:
#機械学習の実行関数
def get_result_math(df):
    df_pret = make_data_math(df)#旧変数で2020-2021
    model = train_model(df_pret,a=2,b=1,c=None)
    df_train= get_train(df_pret)
    df_test = get_test(df_pret, model)
    df_pred = get_pred(df_pret, model)
    df_res=[df_pret, model, df_train, df_test, df_pred]
    return df_res

In [27]:
#機械学習の実行
df_res_math = get_result_math(df_math)

In [28]:
#学習値と予測値の説明変数の列の並びが等しいか
#df_res[0]=[X_train,y_train,X_test,y_test,X_test_kikaku,X_pred,df_pred]
df = df_res_math
df[0][0].columns.values == df[0][5].columns.values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [29]:
#説明変数の確認
df_res_math[0][5].columns.values

array(['学年', '（1）実点・立式点', '（1）カッコ点・立式点', '（1）実点・答え点', '（1）カッコ点・答え点',
       '（1）実点・その他', '（1）カッコ点・その他', '（2）実点・立式点', '（2）カッコ点・立式点',
       '（2）実点・答え点', '（2）カッコ点・答え点', '（2）実点・その他', '（2）カッコ点・その他',
       '（3）実点・立式点', '（3）カッコ点・立式点', '（3）実点・答え点', '（3）カッコ点・答え点',
       '（3）実点・その他', '（3）カッコ点・その他', '（1）ポイント数合計', '（2）ポイント数合計',
       '（3）ポイント数合計', 'ポイント採点', '（1）配点', '（2）配点', '（3）配点', '模範解答の記述行数（１）',
       '模範解答の記述行数（２）', '模範解答の記述行数（３）', '在宅勤務', '科目_数学X', '科目_数学X/A',
       '科目_数学X/A・Y/B', '科目_数学Y', '科目_数学Y/B', '科目_数学Z', '分野_いろいろな式',
       '分野_データの分析', '分野_ベクトル', '分野_三角関数', '分野_二次関数', '分野_図形と方程式',
       '分野_図形と計量', '分野_図形の性質', '分野_場合の数と確率', '分野_平面上の曲線と複素数平面',
       '分野_微分法・積分法', '分野_指数関数・対数関数', '分野_数と式', '分野_数列', '分野_数列の極限',
       '分野_整数の性質'], dtype=object)

In [30]:
#データの数
len(df_math)

507

In [31]:
#学習するデータの数
len(df_res_math[0][0])

422

In [32]:
#検証するデータの数
len(df_res_math[0][2])

41

In [33]:
#予測するデータの数
len(df_res_math[0][5])

44

In [34]:
#結果出力
select_cols = ['最終ペース','企画ペース','AI想定ペース','実企誤差','実予誤差','AI乖離度','元の乖離度']
df=df_res_math


name_file = "分野別ペース予測_22秋後半納品_数学_1025_v1" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name_file)) as writer:
    df[2].to_excel(writer, sheet_name='訓練値_数学',encoding='utf-8-sig', index = False)
    df[3].to_excel(writer, sheet_name='検証値_数学',encoding='utf-8-sig', index = False)
    pandas_tool.summary(df[3].loc[:,select_cols])[0].to_excel(writer, sheet_name='検証値要約_数学',encoding='utf-8-sig', index = True)
    df[4].to_excel(writer, sheet_name='予測値_数学',encoding='utf-8-sig', index = False)

## 英語

In [35]:
df_eng.columns.values

array(['年度', '採点回', '学年', '科目', 'ロット番号', '大問', '分野', '解答言語', 'ポイント採点',
       '企画ペース', '最終ペース', 'パターン数', '配点合計', '文字数合計', 'ポイント数合計', '減点要素の数合計',
       '在宅勤務'], dtype=object)

In [36]:
df_eng.dtypes

年度            int64
採点回          object
学年            int64
科目           object
ロット番号        object
大問            int64
分野           object
解答言語         object
ポイント採点        int64
企画ペース       float64
最終ペース       float64
パターン数         int64
配点合計          int64
文字数合計       float64
ポイント数合計     float64
減点要素の数合計      int64
在宅勤務          int64
dtype: object

In [42]:
def make_data_eng(df_input):#秋採点予測
    
    """
    国語モデル用の学習と検証データを作成
    1.データから不要な列を削除
    2.ダミー化
    3.説明変数と目的変数に分ける
    4.学習と検証に分ける
    5.選択用変数を除外し、各データフレームを作成
    6.[X_train,y_train,X_test,y_test,X_test_info]をデータフレームとしてreturn
    """
    df = df_input.copy()

    #1.不要な列削除
    df = df.drop(columns = ['大問','科目'])

    #2.名義変数のエンコーディング pandas get_dummies関数でone hot encording
    df = pd.get_dummies(df, drop_first=True, columns=['解答言語'])
    #df = pd.get_dummies(df, columns=['解答言語'])
    df = pd.get_dummies(df, columns=['分野'])
    df_pred = df.copy()
    df_pred= df_pred[(df_pred['採点回'] == '秋')&(df_pred['年度']==2022)]#納品する用の予測値算出に使うデータ。納品データの型にも使う。
    df = df[~((df['採点回'] == '秋')&(df['年度']==2022))]#モデル作成と検証に使うデータ。22秋を除いて作成。
    #3.上記のデータセットを説明変数と目的変数で分ける
    df_X = df.drop(columns = {"最終ペース"})#目的変数を除外した（説明変数と選択用変数だけ含む）データフレーム
    df_y = df.loc[:,['年度','採点回','最終ペース']]#目的変数と選択用変数だけ含むデータフレーム
    
    #4.データフレームの分離
    """秋冬が訓練用"""
    #訓練用説明変数
    X_train = df_X[~((df_X['採点回'] == '夏')&(df_X['年度']==2022))]
    #訓練用目的変数
    y_train = df_y[~((df_y['採点回'] == '夏')&(df_y['年度']==2022))]

    """22夏が検証用"""
    #訓練用説明変数
    X_test = df_X[(df_X['採点回'] == '夏')&(df_X['年度']==2022)]
    #訓練用目的変数
    y_test = df_y[(df_y['採点回'] == '夏')&(df_y['年度']==2022)]
    """22秋が納品用"""
    #訓練用説明変数
    X_pred = df_pred.drop(columns = {"最終ペース","企画ペース",'採点回',"年度",'ロット番号'})#目的変数を除外した（説明変数と選択用変数だけ含む）データフレーム
    #訓練用目的変数
    #y_pred = df_pred.loc[:,['年度','採点回_夏','最終ペース']]#目的変数と選択用変数だけ含むデータフレーム
    
    #5.不要な変数を削除
    rem_cols_x = ["企画ペース","採点回","年度","ロット番号"]
    rem_cols_y = ["採点回","年度"]

    #説明変数
    X_train = X_train.drop(columns = rem_cols_x)
    X_test_kikaku =X_test.loc[:,['企画ペース']]
    X_test = X_test.drop(columns = rem_cols_x)
         
    #目的変数
    y_train = y_train.drop(columns = rem_cols_y)
    y_test = y_test.drop(columns = rem_cols_y)
    
    #各モデル用データを各データフレームにまとめ、それらをさらにデータフレームにまとめて返す。
    df_res=[X_train,y_train,X_test,y_test,X_test_kikaku,X_pred,df_pred]

    return df_res

## 結果出力

In [43]:
#機械学習の実行関数
def get_result_eng(df):
    df_pret = make_data_eng(df)#旧変数で2020-2021
    model = train_model(df_pret,a=2,b=1,c=None)
    df_train= get_train(df_pret)
    df_test = get_test(df_pret, model)
    df_pred = get_pred(df_pret, model)
    df_res=[df_pret, model, df_train, df_test, df_pred]
    return df_res

In [44]:
#機械学習の実行
df_res_eng = get_result_eng(df_eng)

In [45]:
#学習値と予測値の説明変数の列の並びが等しいか
#df_res[0]=[X_train,y_train,X_test,y_test,X_test_kikaku,X_pred,df_pred]
df = df_res_eng
df[0][0].columns.values == df[0][5].columns.values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [46]:
#説明変数の確認
df_res_eng[0][5].columns.values

array(['学年', 'ポイント採点', 'パターン数', '配点合計', '文字数合計', 'ポイント数合計', '減点要素の数合計',
       '在宅勤務', '解答言語_英語', '分野_リスニング', '分野_内容説明', '分野_和訳', '分野_自由英作文',
       '分野_英作文', '分野_英作文（単語）', '分野_英訳', '分野_要約'], dtype=object)

In [47]:
#データの数
len(df_eng)

382

In [48]:
#学習するデータの数
len(df_res_eng[0][0])

321

In [49]:
#検証するデータの数
len(df_res_eng[0][2])

31

In [50]:
#予測するデータの数
len(df_res_eng[0][5])

30

In [51]:
df[3].columns

Index(['学年', 'ポイント採点', 'パターン数', '配点合計', '文字数合計', 'ポイント数合計', '減点要素の数合計', '在宅勤務',
       '解答言語', '分野', 'AI想定ペース', '最終ペース', '企画ペース', '実企誤差', '実予誤差', 'AI乖離度',
       '元の乖離度', '最終ランク', '学習データ_平均値', '学習データ_最大値', '学習データ_最小値', '学習データ_中央値',
       '学習データ_N数'],
      dtype='object')

In [52]:
#結果出力
select_cols = ['最終ペース','企画ペース','AI想定ペース','実企誤差','実予誤差','AI乖離度','元の乖離度']
df=df_res_eng.copy()


name_file = "分野別ペース予測_22秋後半納品_英語_1025_v1" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name_file)) as writer:
    df[2].to_excel(writer, sheet_name='訓練値_英語',encoding='utf-8-sig', index = False)
    df[3].to_excel(writer, sheet_name='検証値_英語',encoding='utf-8-sig', index = False)
    pandas_tool.summary(df[3].loc[:,select_cols])[0].to_excel(writer, sheet_name='検証値要約_英語',encoding='utf-8-sig', index = True)
    df[4].to_excel(writer, sheet_name='予測値_英語',encoding='utf-8-sig', index = False)

In [53]:
#結果出力
select_cols = ['最終ペース','企画ペース','AI想定ペース','実企誤差','実予誤差','AI乖離度','元の乖離度']

name_file = "22秋後半_1028_DA予測値_設問別ペース_手動修正前_v1" #ファイル名
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス
path_file = r'{p}/{n}'.format(p = path_folder, n = name_file)#ファイルパスとファイル名

with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name_file)) as writer:
    df_res_jpn[4].to_excel(writer, sheet_name='予測値_国語',encoding='utf-8-sig', index = False)
    df_res_math[4].to_excel(writer, sheet_name='予測値_数学',encoding='utf-8-sig', index = False)
    df_res_eng[4].to_excel(writer, sheet_name='予測値_英語',encoding='utf-8-sig', index = False)

# 以下はメモ

In [36]:
#重要度出力
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス


res = get_importance(df_pret_eng, model_eng)
name = "分野別ペース予測重要度_英語_22夏検証_0905_v1"#ファイル名
path_file = r'{p}/{n}'.format(p = path_folder, n = name)#ファイルパスとファイル名
res.savefig('{}/{}.png'.format(path_folder,name), format="png", dpi=300)

In [37]:
#樹形図出力
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス


treegraph = make_treegraph(df_pret_eng, model_eng)
name = "分野別ペース予測樹形図_英語_22夏検証_0905_v1" #ファイル名
path_file = r'{p}/{n}'.format(p = path_folder, n = name)#ファイルパスとファイル名
treegraph.render(path_file)

'/Users/s.ogura/Documents/CRLEA/data/output/分野別ペース予測樹形図_英語_22夏検証_0905_v1.pdf'

# 各科目の予測に用いた学年、分野、ランク別件数

In [47]:
df = df_jpn.copy()
df = df.loc[:,['年度','採点回','学年','分野','最終ペース']]#目的変数と選択用変数だけ含むデータフレーム

x = "最終ペース"
conditions = [
        (df[x] >= 600),
        (df[x] >= 500),
        (df[x] >= 400),
        (df[x] >= 300),
        (df[x] >= 200),
        (df[x] >= 100),
        (df[x] >= 0)
         ]
choices = ["600~", "500~600", "400~500", "300~400", "200~300", "100~200","0~100"]
df.loc[:,'最終ランク'] = np.select(conditions, choices, default = 0)

#3.上記のデータセットを説明変数と目的変数で分ける
df_y = df[(df['採点回'] == '夏')&(df['年度']==2021)]
df_X = df[~((df['採点回'] == '夏')&(df['年度']==2021))]

df_y = df_y.groupby(['学年','最終ランク','分野'],as_index=False).count()
df_y = df_y.sort_values(['分野','学年','最終ランク'])
df_y = df_y.rename(columns = {'年度':'予測データの件数'})
res_y = df_y.drop(columns = ['採点回','最終ペース'])

                            
df_X = df_X.groupby(['学年','最終ランク','分野'],as_index=False).count()
df_X = df_X.sort_values(['分野','学年','最終ランク'])
df_X = df_X.rename(columns = {'年度':'学習データの件数'})
res_X = df_X.drop(columns = ['採点回','最終ペース'])

res_j = pandas_tool.merge(res_y, res_X,key=['学年','最終ランク','分野'],how='left',view=True)
res_j = res_j.fillna({'学習データの件数': 0})
res_j['科目']='国語'

df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:25
df2のオブザベーション:90
mergeのオブザベーション:25


In [43]:
df = df_math.copy()
df = df.loc[:,['年度','採点回','学年','分野','最終ペース']]#目的変数と選択用変数だけ含むデータフレーム

x = "最終ペース"
conditions = [
        (df[x] >= 600),
        (df[x] >= 500),
        (df[x] >= 400),
        (df[x] >= 300),
        (df[x] >= 200),
        (df[x] >= 100),
        (df[x] >= 0)
         ]
choices = ["600~", "500~600", "400~500", "300~400", "200~300", "100~200","0~100"]
df.loc[:,'最終ランク'] = np.select(conditions, choices, default = 0)

#3.上記のデータセットを説明変数と目的変数で分ける
df_y = df[(df['採点回'] == '夏')&(df['年度']==2021)]
df_X = df[~((df['採点回'] == '夏')&(df['年度']==2021))]

df_y = df_y.groupby(['学年','最終ランク','分野'],as_index=False).count()
df_y = df_y.sort_values(['分野','学年','最終ランク'])
df_y = df_y.rename(columns = {'年度':'予測データの件数'})
res_y = df_y.drop(columns = ['採点回','最終ペース'])

                            
df_X = df_X.groupby(['学年','最終ランク','分野'],as_index=False).count()
df_X = df_X.sort_values(['分野','学年','最終ランク'])
df_X = df_X.rename(columns = {'年度':'学習データの件数'})
res_X = df_X.drop(columns = ['採点回','最終ペース'])

res_m = pandas_tool.merge(res_y, res_X,key=['学年','最終ランク','分野'],how='left',view=True)
res_m = res_m.fillna({'学習データの件数': 0})
res_m['科目']='数学'

df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:36
df2のオブザベーション:125
mergeのオブザベーション:36


In [44]:
df = df_eng.copy()
df = df.loc[:,['年度','採点回','学年','分野','最終ペース']]#目的変数と選択用変数だけ含むデータフレーム

x = "最終ペース"
conditions = [
        (df[x] >= 600),
        (df[x] >= 500),
        (df[x] >= 400),
        (df[x] >= 300),
        (df[x] >= 200),
        (df[x] >= 100),
        (df[x] >= 0)
         ]
choices = ["600~", "500~600", "400~500", "300~400", "200~300", "100~200","0~100"]
df.loc[:,'最終ランク'] = np.select(conditions, choices, default = 0)

#3.上記のデータセットを説明変数と目的変数で分ける
df_y = df[(df['採点回'] == '夏')&(df['年度']==2021)]
df_X = df[~((df['採点回'] == '夏')&(df['年度']==2021))]

df_y = df_y.groupby(['学年','最終ランク','分野'],as_index=False).count()
df_y = df_y.sort_values(['分野','学年','最終ランク'])
df_y = df_y.rename(columns = {'年度':'予測データの件数'})
res_y = df_y.drop(columns = ['採点回','最終ペース'])

                            
df_X = df_X.groupby(['学年','最終ランク','分野'],as_index=False).count()
df_X = df_X.sort_values(['分野','学年','最終ランク'])
df_X = df_X.rename(columns = {'年度':'学習データの件数'})
res_X = df_X.drop(columns = ['採点回','最終ペース'])

res_e = pandas_tool.merge(res_y, res_X,key=['学年','最終ランク','分野'],how='left',view=True)
res_e = res_e.fillna({'学習データの件数': 0})
res_e['科目']='英語'

df1のキー重複
0
df2のキー重複
0
df1のオブザベーション:23
df2のオブザベーション:64
mergeのオブザベーション:23


In [46]:
"""
Excelの書き出し 
"""

df1 = res_j
df2 = res_m
df3 = res_e

name = "分野別ペース予測8月モデル_学習検証データ件数"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/output"#フォルダパス

#Excel
with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name)) as writer:
    df1.to_excel(writer, sheet_name='国語',encoding='utf-8-sig', index = False)
    df2.to_excel(writer, sheet_name='数学',encoding='utf-8-sig', index = False)
    df3.to_excel(writer, sheet_name='英語',encoding='utf-8-sig', index = False)